In [1]:
from IPython.display import display
import pandas as pd

# debug best20GTEx_TCGA.sh.out LUAD, LUSC
UNEXPECTED RESULTS" There are only three interesections that contain Spleen and LUAD. each of these intersections is composed of many other sets. There are only a hand full of genes shared between LUAD and Spleen. Are the samples mislabled? Quick test: drop these genes?

[ ‘DES’], [ ‘KRT6A’], [ ‘CLU’]

60/83 gene signatures have DES in their top 20
47/83 gene signatures have KRTGA in their top 20
25/83 gene signatures have CLU in their top 20

In [2]:
deseqResultsDir="/private/groups/kimlab/aedavids/deconvolution/best20GTEx_TCGA/trainingSet/best20GTEx_TCGA.sh.out/GTEx_TCGA-design-tilda_gender_category-padj-0001-lfc-20-n-20"

In [3]:
LUADResultsFile = ! ls $deseqResultsDir/LUAD_vs_all.results
LUADResultsFile = LUADResultsFile[0]
LUADResultsFile

'/private/groups/kimlab/aedavids/deconvolution/best20GTEx_TCGA/trainingSet/best20GTEx_TCGA.sh.out/GTEx_TCGA-design-tilda_gender_category-padj-0001-lfc-20-n-20/LUAD_vs_all.results'

In [4]:
SpleenFile = ! ls $deseqResultsDir/Spleen_vs_all.results
SpleenFile = SpleenFile[0]
SpleenFile

'/private/groups/kimlab/aedavids/deconvolution/best20GTEx_TCGA/trainingSet/best20GTEx_TCGA.sh.out/GTEx_TCGA-design-tilda_gender_category-padj-0001-lfc-20-n-20/Spleen_vs_all.results'

In [5]:
!head -n 2 $SpleenFile

name,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
MT-RNR2,774541.966031772,-2.20309186124905,0.13635013080831,-16.1576072438559,1.00383004783422e-58,1.13396873619954e-57


In [6]:
spleenGenes = ! cut -d , -f 1 $SpleenFile
spleenGenes[0:5]

['name', 'MT-RNR2', 'MT-RNR1', 'FN1', 'DES']

In [7]:
LUADGenes = ! cut -d , -f 1 $LUADResultsFile
LUADGenes[0:5]

['name', 'DES', 'CLU', 'KRT6A', 'IGHA1']

In [8]:
spleenGenesSet = set(spleenGenes)
LUADGenesSet = set( LUADGenes )
spleenGenesSet.intersection( LUADGenesSet )

{'CLU', 'DES', 'KRT6A', 'name'}

# Explore CIBERSORTx results

In [9]:
CIBERSORTxOutDir="/private/groups/kimlab/aedavids/deconvolution/best20GTEx_TCGA/trainingSet/best20GTEx_TCGA.sh.out/CIBERSORTxFractionsWorkflow.wdl.output"
fractionsFile = CIBERSORTxOutDir + "/results.txt"
fractionsDF = pd.read_csv(fractionsFile, sep="\t")
print(fractionsDF.columns)
print(fractionsDF.shape)
fractionsDF.iloc[0:5,0:2]

Index(['Mixture', 'ACC', 'Adipose_Subcutaneous', 'Adipose_Visceral_Omentum',
       'Adrenal_Gland', 'Artery_Aorta', 'Artery_Coronary', 'Artery_Tibial',
       'BLCA', 'BRCA', 'Bladder', 'Brain_Amygdala',
       'Brain_Anterior_cingulate_cortex_BA24', 'Brain_Caudate_basal_ganglia',
       'Brain_Cerebellar_Hemisphere', 'Brain_Cerebellum', 'Brain_Cortex',
       'Brain_Frontal_Cortex_BA9', 'Brain_Hippocampus', 'Brain_Hypothalamus',
       'Brain_Nucleus_accumbens_basal_ganglia', 'Brain_Putamen_basal_ganglia',
       'Brain_Spinal_cord_cervical_c-1', 'Brain_Substantia_nigra',
       'Breast_Mammary_Tissue', 'CESC', 'CHOL', 'COAD',
       'Cells_Cultured_fibroblasts', 'Cells_EBV-transformed_lymphocytes',
       'Cervix_Endocervix', 'Colon_Sigmoid', 'Colon_Transverse', 'DLBC',
       'ESCA', 'Esophagus_Gastroesophageal_Junction', 'Esophagus_Mucosa',
       'Esophagus_Muscularis', 'GBM', 'HNSC', 'Heart_Atrial_Appendage',
       'Heart_Left_Ventricle', 'KICH', 'KIRC', 'KIRP', 'Kidney_Cortex'

,Mixture,ACC
0,GTEX-1117F-0226-SM-5GZZ7,0.000000
1,GTEX-1117F-0526-SM-5EGHJ,0.005691
2,GTEX-1117F-0726-SM-5GIEN,0.019295
3,GTEX-1117F-2826-SM-5GZXL,0.000000
4,GTEX-1117F-3226-SM-5N9CT,0.003739


In [10]:
dataDir = "/private/home/aedavids/extraCellularRNA/deconvolutionAnalysis/data/metrics/exploreClassificationErrors.best20GTEx_TCGA.FN.LUAD.out"
LUADFalseNegativeFile = dataDir + "/falseNegative.csv"
LUADFalseNegativesDF = pd.read_csv(LUADFalseNegativeFile)
print(f'LUADFalseNegativesDF.shape : {LUADFalseNegativesDF.shape}')
LUADFalseNegativesDF.head()

LUADFalseNegativesDF.shape : (171, 2)


,Mixture,predictedCat
0,LUAD-05-4250-TP,CHOL
1,LUAD-05-4382-TP,Spleen
2,LUAD-05-4389-TP,DLBC
3,LUAD-05-4395-TP,UCEC
4,LUAD-05-4397-TP,UCEC


In [11]:
# select LUAD false Negatives CIBERSORTx fractions
#LUAD_FN_FractionsDF = fractionsDF.loc[ LUADFalseNagtiveDF.loc[:, "Mixture"], :]
FNSamples = LUADFalseNegativesDF.loc[:, "Mixture"].values
FNSamples[0:4]

array(['LUAD-05-4250-TP', 'LUAD-05-4382-TP', 'LUAD-05-4389-TP',
       'LUAD-05-4395-TP'], dtype=object)

In [12]:
#LUADFalseNagtiveDF.set_index('Mixture')
selectRows = fractionsDF.loc[:, "Mixture"].isin(FNSamples)
print(f'sum(selectRows) : {sum(selectRows)} len(selectRows) : {len(selectRows)}')
print(f'type(selectRows) : {type(selectRows)}')
print(selectRows.head())
LUAD_FN_FractionsDF = fractionsDF.loc[ selectRows, :]
print(f'LUAD_FN_FractionsDF.shape : {LUAD_FN_FractionsDF.shape}')
LUAD_FN_FractionsDF.iloc[0:5, 0:5]

sum(selectRows) : 171 len(selectRows) : 15801
type(selectRows) : <class 'pandas.core.series.Series'>
0    False
1    False
2    False
3    False
4    False
Name: Mixture, dtype: bool
LUAD_FN_FractionsDF.shape : (171, 87)


,Mixture,ACC,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland
13312,LUAD-05-4250-TP,0.010323,0.033233,0.000000,0.000000
13313,LUAD-05-4382-TP,0.016655,0.005405,0.017205,0.000000
13315,LUAD-05-4389-TP,0.013180,0.038823,0.001671,0.008196
13316,LUAD-05-4395-TP,0.000000,0.000000,0.000583,0.010407
13317,LUAD-05-4397-TP,0.039333,0.027158,0.008253,0.000880


### Make it easier to debug 
We use the max fraction to determin the category for each sample. Set factions <= 0.1 == 0.0 The drop columns that sum to zero

In [13]:
# select numeric columns
mixtureDF = LUAD_FN_FractionsDF.loc[:, ['Mixture']]

xxxDF = LUAD_FN_FractionsDF.iloc[:, 1:]
display(xxxDF.iloc[0:5, 0:5] )
# where : Where cond is True, keep the original value. Where False, 
# replace with corresponding value from other.
# xxxDF.where( xxxDF >= 0.1, 0.0, inplace=True)
# xxxDF.where( xxxDF >= 0.30, 0.0, inplace=True)
xxxDF.where( xxxDF >= 0.20, 0.0, inplace=True)

byCols = 1
simplifiedFN_DF = pd.concat( [mixtureDF, xxxDF], axis=byCols )
print('\n concat')
print(simplifiedFN_DF.shape)
display(simplifiedFN_DF.iloc[0:5, 0:6])

,ACC,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta
13312,0.010323,0.033233,0.000000,0.000000,0.000000
13313,0.016655,0.005405,0.017205,0.000000,0.011135
13315,0.013180,0.038823,0.001671,0.008196,0.000000
13316,0.000000,0.000000,0.000583,0.010407,0.000000
13317,0.039333,0.027158,0.008253,0.000880,0.000000



 concat
(171, 87)


,Mixture,ACC,Adipose_Subcutaneous,Adipose_Visceral_Omentum,Adrenal_Gland,Artery_Aorta
13312,LUAD-05-4250-TP,0.0,0.0,0.0,0.0,0.0
13313,LUAD-05-4382-TP,0.0,0.0,0.0,0.0,0.0
13315,LUAD-05-4389-TP,0.0,0.0,0.0,0.0,0.0
13316,LUAD-05-4395-TP,0.0,0.0,0.0,0.0,0.0
13317,LUAD-05-4397-TP,0.0,0.0,0.0,0.0,0.0


In [14]:
# https://stackoverflow.com/a/23573084/4586180
# df.loc[(df.sum(axis=1) != 0), (df.sum(axis=0) != 0)]
# display(simplifiedFN_DF.sum() )
# print()

defaultMaxRows = pd.get_option('display.max_rows')
pd.set_option('display.max_rows', None)
display( simplifiedFN_DF.loc[ :, (simplifiedFN_DF.sum(axis=0) != 0) ] )
pd.set_option('display.max_rows', defaultMaxRows)

,Mixture,DLBC,LUSC,Lung,Spleen,Correlation,RMSE
13312,LUAD-05-4250-TP,0.000000,0.000000,0.000000,0.000000,0.665453,0.768459
13313,LUAD-05-4382-TP,0.000000,0.000000,0.000000,0.245565,0.487115,0.872455
13315,LUAD-05-4389-TP,0.000000,0.000000,0.000000,0.000000,0.739077,0.693320
13316,LUAD-05-4395-TP,0.000000,0.000000,0.000000,0.000000,0.596106,0.804612
13317,LUAD-05-4397-TP,0.000000,0.000000,0.000000,0.000000,0.854824,0.566931
13318,LUAD-05-4398-TP,0.000000,0.000000,0.000000,0.206207,0.732834,0.721772
13319,LUAD-05-4402-TP,0.000000,0.000000,0.000000,0.000000,0.764405,0.693801
13320,LUAD-05-4405-TP,0.000000,0.000000,0.000000,0.000000,0.783286,0.661397
13321,LUAD-05-4410-TP,0.000000,0.000000,0.000000,0.323668,0.681428,0.758616
13323,LUAD-05-4418-TP,0.000000,0.000000,0.000000,0.273719,0.540615,0.842024


In [15]:
pd.get_option('display.max_rows')


60

In [16]:
fdlkgdlkfl

NameError: name 'fdlkgdlkfl' is not defined

In [ ]:
# df.where(df <= 9, 11, inplace=True)
# skip the Mixture column. 
# xxxDF = LUAD_FN_FractionsDF.iloc[:, 1:].where( LUAD_FN_FractionsDF <= 0.1, 0.0)
# xxxDF = LUAD_FN_FractionsDF.where( LUAD_FN_FractionsDF.iloc[:, 1:] <= 0.1, 999.0)
#xxxDF = LUAD_FN_FractionsDF.iloc[:, 1:] <= 0.1, 999.0)
xxxDF.iloc[0:5, 0:5]

In [ ]:
yyyDF = LUAD_FN_FractionsDF.iloc[:, 1:].copy()
print(yyyDF.shape)
# zzzDF = yyyDF.where( yyyDF <= 0.1, 999.0)
# zzzDF.iloc[0:5, 0:5]
yyyDF.where( yyyDF <= 0.1, 999.0, inplace=True)
yyyDF.iloc[0:5, 0:5]

In [ ]:
fooDF = pd.DataFrame({'a': [0, 5, 10, 15], 'b' : [1, 6, 11, 16]})
print(fooDF)
print()
# where : Where cond is True, keep the original value. Where False, 
# replace with corresponding value from other.
#
fooDF.where( fooDF >= 5, 999 )
# fooDF

In [ ]:
# select numeric columns
xxxDF = LUAD_FN_FractionsDF.iloc[:, 1:]
print( xxxDF.iloc[0:5, 0:4] )
print()
yyyDF = xxxDF.where( xxxDF >= 0.02, 999.0)
print(yyyDF.shape)
yyyDF.iloc[0:5, 0:4]